In [47]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import os
import glob
from functools import partial

plt.style.use('ggplot')

In [5]:
!pwd

/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1


In [35]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/Michael")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)
combined_csv.to_csv("combined_Michael.csv", index=False)

In [36]:
mj_df = pd.read_csv('combined_Michael.csv')

In [37]:
mj_df.head(5)

,Unnamed: 0,Rk,G,Date,Age,Tm,,Opp,.1,GS,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,Unnamed: 0.1,Unnamed: 0.1.1
0,1,1,1,1991-11-01,28-257,CHI,NaN,PHI,W (+20),1,...,4,3,1,2,2,26,22.6,NaN,NaN,NaN
1,2,2,2,1991-11-02,28-258,CHI,@,MIL,L (-2),1,...,3,5,1,4,1,46,37.6,NaN,NaN,NaN
2,3,3,3,1991-11-05,28-261,CHI,NaN,GSW,L (-8),1,...,4,1,1,5,5,40,30.6,NaN,NaN,NaN
3,4,4,4,1991-11-06,28-262,CHI,@,BOS,W (+19),1,...,6,1,1,2,2,44,36.2,NaN,NaN,NaN
4,5,5,5,1991-11-08,28-264,CHI,NaN,DAL,W (+16),1,...,5,4,0,0,1,26,28.3,NaN,NaN,NaN


In [41]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/Lebron")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)
combined_csv.to_csv("combined_Lebron.csv", index=False)

In [42]:
lbj_df = pd.read_csv('combined_Lebron.csv')

### Turn specific columns to numeric instead of string

In [84]:
lbj_cols = list(lbj_df.columns)
print(lbj_cols)
    
num_cols = ['GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
            'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
            'TOV', 'PF', 'PTS', 'GmSc', '+/-']

lbj_df[num_cols] = lbj_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)
mj_df[num_cols] = mj_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)

['Unnamed: 0', 'Rk', 'G', 'Date', 'Age', 'Tm', '\xa0', 'Opp', '\xa0.1', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']


In [92]:
mj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3363 entries, 0 to 3362
Data columns (total 33 columns):
Unnamed: 0        3363 non-null int64
Rk                3363 non-null object
G                 3363 non-null object
Date              3363 non-null object
Age               3363 non-null object
Tm                3363 non-null object
                  1755 non-null object
Opp               3363 non-null object
 .1               3363 non-null object
GS                3216 non-null float64
MP                0 non-null float64
FG                3216 non-null float64
FGA               3216 non-null float64
FG%               3216 non-null float64
3P                3216 non-null float64
3PA               3216 non-null float64
3P%               2226 non-null float64
FT                3216 non-null float64
FTA               3216 non-null float64
FT%               3126 non-null float64
ORB               3216 non-null float64
DRB               3216 non-null float64
TRB               3216 non

### Clean up and format LBJ and MJ df's
- Clean up rows that are repeated headers
- Convert `Date` column into datetime format
- Drop `Unnamed: 0` column from both df's

In [100]:
#Run this after cleaning rows
lbj_df['Date'] = lbj_df['Date'].apply(pd.to_datetime, format='%Y-%m-%d')
mj_df['Date'] = mj_df['Date'].apply(pd.to_datetime, format='%Y-%m-%d')

/Users/edwardkim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [101]:
lbj_df = lbj_df[lbj_df.Rk != 'Rk']
mj_df = mj_df[mj_df.Rk != 'Rk']

In [106]:
lbj_df.columns

Index(['Unnamed: 0', 'Rk', 'G', 'Date', 'Age', 'Tm', ' ', 'Opp', ' .1', 'GS',
       'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-'],
      dtype='object')

In [116]:
lbj_df.drop(columns=['Unnamed: 0'], inplace=True)
mj_df.drop(columns=['Unnamed: 0'], inplace=True)

In [198]:
lbj_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1198 entries, 0 to 1259
Data columns (total 30 columns):
Rk      1198 non-null object
G       1198 non-null object
Date    1198 non-null datetime64[ns]
Age     1198 non-null object
Tm      1198 non-null object
        593 non-null object
Opp     1198 non-null object
 .1     1198 non-null object
GS      1198 non-null float64
MP      0 non-null float64
FG      1198 non-null float64
FGA     1198 non-null float64
FG%     1198 non-null float64
3P      1198 non-null float64
3PA     1198 non-null float64
3P%     1158 non-null float64
FT      1198 non-null float64
FTA     1198 non-null float64
FT%     1185 non-null float64
ORB     1198 non-null float64
DRB     1198 non-null float64
TRB     1198 non-null float64
AST     1198 non-null float64
STL     1198 non-null float64
BLK     1198 non-null float64
TOV     1198 non-null float64
PF      1198 non-null float64
PTS     1198 non-null float64
GmSc    1198 non-null float64
+/-     1198 non-null float

In [205]:
lbj_df.head()

,Rk,G,Date,Age,Tm,,Opp,.1,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1,2008-10-28,23-303,CLE,@,BOS,L (-5),1.0,NaN,...,6.0,7.0,6.0,2.0,1.0,3.0,4.0,22.0,14.1,-3.0
1,2,2,2008-10-30,23-305,CLE,NaN,CHA,W (+17),1.0,NaN,...,7.0,9.0,9.0,0.0,1.0,5.0,3.0,22.0,17.0,10.0
2,3,3,2008-11-01,23-307,CLE,@,NOH,L (-12),1.0,NaN,...,5.0,7.0,13.0,3.0,0.0,5.0,3.0,15.0,15.3,-8.0
3,4,4,2008-11-03,23-309,CLE,@,DAL,W (+19),1.0,NaN,...,6.0,8.0,3.0,2.0,1.0,1.0,1.0,29.0,24.0,13.0
4,5,5,2008-11-05,23-311,CLE,NaN,CHI,W (+14),1.0,NaN,...,7.0,9.0,6.0,4.0,0.0,5.0,1.0,41.0,36.0,22.0


## Advanced Stats and Reg Season Stats For All Players

In [143]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/advanced_stats")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)
combined_csv.to_csv("advanced_85-19.csv", index=False)

In [144]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/reg_season_stats")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)
combined_csv.to_csv("reg_szn_85-19.csv", index=False)

In [148]:
adv_df = pd.read_csv('../advanced_stats/advanced_85-19.csv')

In [150]:
szn_df = pd.read_csv('reg_szn_85-19.csv')

In [169]:
adv_df.head()

,Unnamed: 0,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,...,,OWS,DWS,WS,WS/48,.1,OBPM,DBPM,BPM,VORP
0,1,1,Álex Abrines,SG,24.0,OKC,75.0,1134.0,9.0,0.567,...,NaN,1.3,1.0,2.2,0.094,NaN,-1.9,0.4,-1.5,0.1
1,2,2,Quincy Acy,PF,27.0,BRK,70.0,1359.0,8.2,0.525,...,NaN,-0.1,1.1,1.0,0.036,NaN,-2.6,0.1,-2.5,-0.2
2,3,3,Steven Adams,C,24.0,OKC,76.0,2487.0,20.6,0.630,...,NaN,6.7,3.0,9.7,0.187,NaN,1.7,-0.6,1.1,2.0
3,4,4,Bam Adebayo,C,20.0,MIA,69.0,1368.0,15.7,0.570,...,NaN,2.3,1.9,4.2,0.148,NaN,-1.1,0.7,-0.4,0.6
4,5,5,Arron Afflalo,SG,32.0,ORL,53.0,682.0,5.8,0.516,...,NaN,-0.1,0.2,0.1,0.009,NaN,-3.8,-1.5,-5.4,-0.6


In [177]:
adv_cols = adv_df.columns
adv_cols
    
num_cols = ['Age','G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 
            'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
            'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

adv_df[num_cols] = adv_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)

adv_df.drop(columns=['Unnamed: 0'], inplace=True)

/Users/edwardkim/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/edwardkim/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Remove rows that are repeated headers

In [178]:
adv_df = adv_df[adv_df.Rk != 'Rk']
szn_df = szn_df[szn_df.Rk != 'Rk']

In [179]:
szn_cols = szn_df.columns

num_cols = ['Age','G', 'GS', 'MP', 'FG',
            'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
            'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
            'PTS']

szn_df[num_cols] = szn_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)

szn_df.drop(columns=['Unnamed: 0'], inplace=True)

In [180]:
szn_df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Quincy Acy,SF,23.0,TOT,63.0,0.0,13.4,1.0,2.2,...,0.660,1.1,2.3,3.4,0.4,0.4,0.4,0.5,1.9,2.7
1,1,Quincy Acy,SF,23.0,TOR,7.0,0.0,8.7,0.9,2.0,...,0.625,0.7,1.4,2.1,0.6,0.6,0.4,0.3,1.1,2.7
2,1,Quincy Acy,SF,23.0,SAC,56.0,0.0,14.0,1.1,2.3,...,0.667,1.2,2.4,3.6,0.4,0.3,0.4,0.5,2.0,2.7
3,2,Steven Adams,C,20.0,OKC,81.0,20.0,14.8,1.1,2.3,...,0.581,1.8,2.3,4.1,0.5,0.5,0.7,0.9,2.5,3.3
4,3,Jeff Adrien,PF,27.0,TOT,53.0,12.0,18.1,2.7,5.2,...,0.639,1.9,3.8,5.8,0.7,0.5,0.7,0.7,2.0,6.8


In [191]:
szn_grouped = szn_df.groupby("Player")

,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Player,,,,,,,,,,,,,,,,,,,,,
A.C. Green,28.571429,76.071429,52.285714,29.642857,3.571429,7.157143,0.497357,0.092857,0.407143,0.154571,...,0.716143,2.721429,5.035714,7.750000,1.085714,0.857143,0.435714,1.192857,1.957143,9.792857
A.J. English,23.500000,75.500000,9.000000,20.600000,4.050000,9.300000,0.436000,0.050000,0.400000,0.136500,...,0.774000,0.900000,1.200000,2.100000,2.150000,0.400000,0.150000,1.350000,1.900000,9.850000
A.J. Guyton,23.500000,23.500000,3.000000,9.000000,1.000000,3.700000,0.180500,0.500000,1.600000,0.187000,...,0.815000,0.150000,0.350000,0.500000,1.400000,0.350000,0.100000,0.650000,0.250000,2.700000
A.J. Hammons,24.000000,22.000000,0.000000,7.400000,0.800000,1.900000,0.405000,0.200000,0.500000,0.500000,...,0.450000,0.400000,1.300000,1.600000,0.200000,0.000000,0.600000,0.500000,1.000000,2.200000
A.J. Price,26.333333,31.888889,2.777778,13.177778,1.888889,5.066667,0.355222,0.666667,2.322222,0.242889,...,0.632250,0.222222,1.033333,1.266667,1.900000,0.366667,0.022222,0.700000,0.766667,5.088889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Šarūnas Marčiulionis*,28.428571,51.857143,4.285714,21.757143,4.157143,8.342857,0.486000,0.357143,1.042857,0.300000,...,0.767571,0.814286,1.414286,2.242857,2.228571,1.228571,0.085714,2.028571,2.628571,12.314286
Žan Tabak,26.000000,38.500000,15.000000,15.183333,2.233333,4.650000,0.475500,0.000000,0.000000,0.000000,...,0.518833,1.366667,2.083333,3.483333,0.750000,0.350000,0.583333,1.116667,2.533333,5.216667
Žarko Paspalj,23.000000,28.000000,1.000000,6.500000,1.000000,2.800000,0.342000,0.000000,0.000000,0.000000,...,0.818000,0.500000,0.500000,1.100000,0.400000,0.100000,0.300000,0.800000,1.300000,2.600000


In [192]:
szn_df['Player'].count

<bound method Series.count of 0          Quincy Acy
1          Quincy Acy
2          Quincy Acy
3        Steven Adams
4         Jeff Adrien
             ...     
17098    Tyler Zeller
17099      Ante Žižić
17100     Ivica Zubac
17101     Ivica Zubac
17102     Ivica Zubac
Name: Player, Length: 16451, dtype: object>